In [ ]:
# from os import path
from pathlib import Path
import json
import gzip

mixed_files = list(Path("./").glob("**/*json*"))
suffixes = {}
for file in mixed_files:
    if file.suffix not in suffixes:
        suffixes[file.suffix] = 0
    suffixes[file.suffix] += 1
suffixes
# looks like no jsonl files 

{'.json': 956, '.gz': 45}

In [27]:
# let's sample a few
content = []
for file in mixed_files:
    print(file)
    if file.suffix == ".json":
        content = file.read_text()
        print(content)
        # content.append(json.loads(file.read_text()))


chat_0012.json
{"id": "c12_0", "source": "kb_import", "lang": "en", "messages": [{"role": "system", "content": "You are a helpful assistant."}, {"role": "user", "content": "install explain brown lazy guide how fox ubuntu dog to"}, {"role": "assistant", "content": "windows windows thanks install fox the api use the install configure function data function linux how use use how over use to jumps linux how configure ubuntu ubuntu how api over linux thanks please dog quick use to windows help quick thanks to data dog over lazy lazy the step dog lazy thanks windows to how explain thanks ubuntu example quick fox lazy install help quick fox linux explain jumps how example ubuntu example over configure function help api over help please api configure install install ubuntu api please jumps lazy use step ubuntu quick"}], "timestamp": "2024-04-29T20:20:00", "rating": null, "_pair_hash": "97b15e3c034dd8087d75e775eb098695"}
{"id": "c12_1", "source": "synthetic_assistant", "lang": "de", "prompt": "

In [ ]:

# jsonl and json with array

json_l_file = "chat_0012.json"
json_file = "chat_0911.json"

with open(json_file, mode = "r") as f:
    json_obj = json.loads(f.read())

json_objs = []
with open(json_l_file, mode = "r") as f:
    for line in f:
        if line.strip():
            json_objs.append(json.loads(line))

In [26]:
json_obj[3]

{'id': 'c911_3',
 'source': 'synthetic_assistant',
 'lang': 'en',
 'prompt': 'please step guide step step windows how quick help linux',
 'response': 'configure please quick linux example guide fox ubuntu api configure example dog quick linux thanks the fox api step ubuntu to api api lazy quick how dog windows fox fox api the install quick function over',
 'timestamp': '2024-04-03T21:34:00',
 'rating': None,
 '_pair_hash': '10af195f583bbd44f59e0906988885aa'}

In [24]:
json_objs[0]

{'id': 'c12_0',
 'source': 'kb_import',
 'lang': 'en',
 'messages': [{'role': 'system', 'content': 'You are a helpful assistant.'},
  {'role': 'user',
   'content': 'install explain brown lazy guide how fox ubuntu dog to'},
  {'role': 'assistant',
   'content': 'windows windows thanks install fox the api use the install configure function data function linux how use use how over use to jumps linux how configure ubuntu ubuntu how api over linux thanks please dog quick use to windows help quick thanks to data dog over lazy lazy the step dog lazy thanks windows to how explain thanks ubuntu example quick fox lazy install help quick fox linux explain jumps how example ubuntu example over configure function help api over help please api configure install install ubuntu api please jumps lazy use step ubuntu quick'}],
 'timestamp': '2024-04-29T20:20:00',
 'rating': None,
 '_pair_hash': '97b15e3c034dd8087d75e775eb098695'}

In [269]:
from json import JSONDecodeError


# id, source, lang, response, timestmap, rating, pair_hash

# either prompt + response
# or messages array


def infer_file_type(filepath):
    # print("processing", filepath)

    if filepath.suffix == ".gz":
        opener = gzip.open(filepath, mode = "rt")
    else:
        opener = open(filepath, mode = "r")
    json_obj = []
    with opener as f:
        # now, detect whether it is json array still or jsonl style

        # try to read the file first, (assume its json)
        try:
            content = f.read()
            json_obj = json.loads(content)
            # this should already be a list / array
            # we need to add source to each one?
            # print(json_obj)
            json_obj = [{**obj, "_file": str(filepath)} for obj in json_obj]
            json_obj = [{**obj, "_row": i} for i, obj in enumerate(json_obj)]
        except JSONDecodeError:
            # treat it like a jsonl file
            try:
                # go back to the beginning
                f.seek(0)
                idx = 0
                for line in f:
                    if line.strip():
                        json_line = json.loads(line)
                        json_line['_file'] = str(filepath)
                        json_line['_row'] = idx
                        idx += 1
                        json_obj.append(json_line)
            except JSONDecodeError:
                print("file", filepath, "is invalid")
    return json_obj

def normalize_record(record):
    # get a dict give a dict
    if "messages" in record:
        # find the last user and asisstant turn
        msgs = record["messages"]
        user_turn = next((message.get("content", "") for message in msgs if message.get("role", "") == "user"), "")
        assistant_turn = next((message.get("content", "") for message in reversed(msgs) if message.get("role", "") == "assistant"), "")
        record["text"] = user_turn + "\n" + assistant_turn
        return record
    prompt = record.get("prompt", "")
    response = record.get("response", "")
    record["text"] = response if response else prompt
    return record

def validate_and_parse_json_array(json_list):
    # validate that one of two cases is true: 1) prompot + response 2) messages array in a specific format
    # each should also be a list of dicts
    if not isinstance(json_list, list):
        return []
    if not isinstance(json_list[0], dict):
        return []
        # id, source, lang, response, timestmap, rating, pair_hash
    # normalize record
    normalized_recs = []
    for record in json_list:
        normalized_recs.append(normalize_record(record))
    # should we validate here?
    return normalized_recs
    

mixed_files = list(Path("./").glob("**/chat*json*"))
sample_size = len(mixed_files)
print(sample_size)
sample_files = [infer_file_type(mixed_files[i]) for i in range(sample_size)]
valid_values = [validate_and_parse_json_array(sample_files[i]) for i in range(sample_size)]
# count the rows and number of files
# lets get this into a pandas dataframe
import pandas as pd
# first, make everything a list of dicts instead of a list of a lsit of dicts
# print(valid_values)
res = []
for rows in valid_values:
    res.extend(rows)
# print(res)
df = pd.DataFrame(res)
# print(df)
n_rows = len(res)
# print(df.columns)
n_files = len(df._file.unique())
n_unique_rows = df.text.nunique()
print("Number of rows: ", n_rows)
print("number of files: ", n_files)
print("number of unique rows:", n_unique_rows)
dup_rate = (n_rows - n_unique_rows) / n_rows
print("duplication rate", dup_rate)

# do it by source...
def calculate_dedup_rate(series):
    n_unique = series.nunique()
    # print("hi", series.name, len(series))
    return (len(series) - n_unique) / len(series)

    # return a reference to a singleton tokenizer so we dont need to keep reloading
import math
import tiktoken
enc = tiktoken.encoding_for_model("gpt-4o")

def get_n_tokens(text):
    return (len(text or "") + 3) // 4  # CORRECT


dedup_by_source = df.groupby("source")[['text']].agg(calculate_dedup_rate)
df['n_tokens'] = df['text'].apply(lambda x: get_n_tokens(x))
from matplotlib import pyplot as plt

quantiles = df[['n_tokens']].describe(percentiles = [.25,.50,.75,0.95])

total_tokens = df['n_tokens'].aggregate(["sum"])
# share of tokens in the 95th + percentile
high_pctile = quantiles.loc['95%', 'n_tokens']
top_rows = df[df['n_tokens'] >= high_pctile]
frac = sum(top_rows['n_tokens']) / total_tokens
frac

# email_pii_filter = "blank@blank.com"

# phone_number_pii_filter = "+1-xxx-xxx-xxxx"
import re
RE_EMAIL = re.compile(r"[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Za-z]{2,}")
RE_PHONE = re.compile(r"\+?\d[\d\-\s]{7,}\d")
df["has_email"] = df["text"].str.contains(RE_EMAIL)
df["has_phone"] = df["text"].str.contains(RE_PHONE)
print("Emails:", int(df["has_email"].sum()), "Phones:", int(df["has_phone"].sum()))
# where + is not necessary, and 1 can be any country code 
# i guess this is really askign what is the area under the curve in the 95th percentile plus?
# put another way, isnt it just 5%
# should give us quantile info + summary stats

1000
Number of rows:  2995
number of files:  1000
number of unique rows: 2745
duplication rate 0.08347245409015025
Emails: 45 Phones: 51


In [266]:
total_tokens = df['n_tokens'].sum()                      # scalar
top_tokens   = top_rows['n_tokens'].sum()                # scalar
frac         = top_tokens / total_tokens                 # scalar float
print(f"Top-5% token share: {frac:.3%}")

Top-5% token share: 24.410%


In [253]:
quantiles.loc['95%', 'n_tokens']

np.float64(306.2999999999997)

In [212]:
import tiktoken
text = pd.Series(["hi bob, how are you !@#$", "I'm good :) "])
enc = tiktoken.get_encoding(encoding_name="cl100k_base")
len(enc.encode_batch(text.values))

2

In [ ]:
enc = tiktoken.get_encoding

KeyError: 'Invalid token for decoding: 100256'

In [139]:
for col in df.columns:
    print(col, type(df[col]))

id <class 'pandas.core.series.Series'>
source <class 'pandas.core.series.Series'>
lang <class 'pandas.core.series.Series'>
messages <class 'pandas.core.series.Series'>
timestamp <class 'pandas.core.series.Series'>
rating <class 'pandas.core.series.Series'>
_pair_hash <class 'pandas.core.series.Series'>
_file <class 'pandas.core.series.Series'>
_row <class 'pandas.core.series.Series'>
text <class 'pandas.core.series.Series'>
prompt <class 'pandas.core.series.Series'>
response <class 'pandas.core.series.Series'>


In [157]:
# Create a sample series to explore
# sample_series = pd.Series([1, 2, 3, 4, 5], index=['a', 'b', 'c', 'd', 'e'])
# print(sample_series)

# Or use one of your columns
your_series = df['source']  # This is a Series

[attr for attr in dir(your_series) if not attr.startswith("_")]

['T',
 'abs',
 'add',
 'add_prefix',
 'add_suffix',
 'agg',
 'aggregate',
 'align',
 'all',
 'any',
 'apply',
 'argmax',
 'argmin',
 'argsort',
 'array',
 'asfreq',
 'asof',
 'astype',
 'at',
 'at_time',
 'attrs',
 'autocorr',
 'axes',
 'backfill',
 'between',
 'between_time',
 'bfill',
 'bool',
 'case_when',
 'clip',
 'combine',
 'combine_first',
 'compare',
 'convert_dtypes',
 'copy',
 'corr',
 'count',
 'cov',
 'cummax',
 'cummin',
 'cumprod',
 'cumsum',
 'describe',
 'diff',
 'div',
 'divide',
 'divmod',
 'dot',
 'drop',
 'drop_duplicates',
 'droplevel',
 'dropna',
 'dtype',
 'dtypes',
 'duplicated',
 'empty',
 'eq',
 'equals',
 'ewm',
 'expanding',
 'explode',
 'factorize',
 'ffill',
 'fillna',
 'filter',
 'first',
 'first_valid_index',
 'flags',
 'floordiv',
 'ge',
 'get',
 'groupby',
 'gt',
 'hasnans',
 'head',
 'hist',
 'iat',
 'idxmax',
 'idxmin',
 'iloc',
 'index',
 'infer_objects',
 'info',
 'interpolate',
 'is_monotonic_decreasing',
 'is_monotonic_increasing',
 'is_unique',

In [164]:
your_series.name

'source'

In [124]:
type([]), isinstance([], list), [][0]

IndexError: list index out of range

In [104]:
import json
filename = "test.json"

test = '[{"id": 1}, {"id":2}]'
json_obj = json.loads(test)
json_obj = [{**obj, "_file": filename} for obj in json_obj]
json_obj = [{**obj, "_row": idx} for idx, obj in enumerate(json_obj)]
json_obj

[{'id': 1, '_file': 'test.json', '_row': 0},
 {'id': 2, '_file': 'test.json', '_row': 1}]

In [ ]:
{**obj, "key": "value"} for obj in json_obj

SyntaxError: invalid syntax (3690124805.py, line 1)

In [222]:
import time
import numpy as np

# Method 1: Just call function on Series (truly batched)
def tokenize_all(text_series):
    texts = text_series.fillna('').tolist()
    return [len(tokens) for tokens in enc.encode_batch(texts)]

# Method 2: Apply row-by-row
def tokenize_single(text):
    return len(enc.encode(text or ''))

# Method 3: Hacky groupby with single group
df['dummy'] = 1
result = df.groupby('dummy')['text'].transform(lambda x: tokenize_all(x))

# Test them
start = time.time()
df['tokens_batch'] = tokenize_all(df['text'])
print(f"Batch: {time.time()-start:.3f}s")

start = time.time()
df['tokens_apply'] = df['text'].apply(tokenize_single)
print(f"Apply: {time.time()-start:.3f}s")

# The groupby trick you mentioned
start = time.time()
df['dummy'] = 1
df['tokens_groupby'] = df.groupby('dummy')['text'].transform(lambda x: tokenize_all(x))
print(f"Groupby hack: {time.time()-start:.3f}s")


Batch: 0.070s
Apply: 0.078s
Groupby hack: 0.072s


In [ ]:
import pandas as pd
import numpy as np
import time
import tiktoken

# Sample data
texts = ['example ' * i for i in range(100, 600)]
df = pd.DataFrame({'text': texts * 2})  # 1000 rows

# Functions
def token_single(text):
    return (len(text or "") + 3) // 4

def token_batch(series):
    return [(len(t or "") + 3) // 4 for t in series]

# Timing
import timeit

# Apply (row-by-row)
time_apply = timeit.timeit(
    lambda: df['text'].apply(token_single),
    number=10
) / 10

# Direct call (batch)  
time_batch = timeit.timeit(
    lambda: token_batch(df['text']),
    number=10
) / 10

# Pipe
time_pipe = timeit.timeit(
    lambda: df['text'].pipe(token_batch),
    number=10
) / 10

print(f"Apply: {time_apply:.6f}s")
print(f"Batch: {time_batch:.6f}s")  
print(f"Pipe:  {time_pipe:.6f}s")
print(f"Batch is {time_apply/time_batch:.1f}x faster")

Apply: 0.000230s
Batch: 0.000080s
Pipe:  0.000087s
Batch is 2.9x faster
